In [1]:
import sys
import os

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize': (9, 6)})
import warnings
warnings.filterwarnings('ignore')

sys.path.append(os.path.join(sys.path[0], '../'))
import utils.preprocessing_utils as pu
import utils.model_fit_utils as mu

# Считка данных и формирование критериев и целевой ф-ции для обучения

In [2]:
events_data = pd.read_csv('../data/train/event_data_train.zip')
submissions_data = pd.read_csv('../data/train/submissions_data_train.zip')

In [3]:
X, y = pu.make_X_y_with_new_fetures(events_data, submissions_data)
X

,days,steps_tried,correct,wrong,correct_ratio,step_correct_ratio,viewed,passed / discovered,passed / started_attempt,passed / viewed
user_id,,,,,,,,,,
2,1.0,2.0,2.0,0.0,1.00,0.830587,9,0.968650,0.938630,0.478740
3,1.0,4.0,4.0,4.0,0.50,0.623948,20,0.962520,0.939309,0.464934
8,1.0,11.0,9.0,21.0,0.30,0.633025,154,0.952075,0.887990,0.411834
14,1.0,1.0,0.0,1.0,0.00,0.558446,9,0.944890,0.943250,0.365771
16,3.0,23.0,23.0,27.0,0.46,0.413285,132,0.933793,0.909504,0.354163
...,...,...,...,...,...,...,...,...,...,...
26774,0.0,0.0,0.0,0.0,0.00,0.000000,1,1.000000,1.000000,0.228748
26781,0.0,0.0,0.0,0.0,0.00,0.000000,6,1.000000,1.000000,0.494669
26788,0.0,0.0,0.0,0.0,0.00,0.000000,1,1.000000,1.000000,0.331215


-**Данне здесь отличаются от данных с baseline.**  
-**Во время формирования датафрейма добавлено подсчет статистики для каждого step_id.**  
-**Так "step_correct_ratio" - отвечает за среднее значение отношения "correct" на "correct + wrong" по всем степам пройденным пользователем**  
-**"passed / discovered", "passed / started_attempt", "passed / viewed" - соответсвующе средние значения для пользователя**

# Обучение моделей

In [4]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [6]:
skf = StratifiedKFold(shuffle=True, random_state=17)

dt_train_test = mu.fit_dt(X_train_scaled, y_train)
dt_grid_serch = mu.fit_grid_search_dt(X_train_scaled, y_train, skf)
lr_train_test = mu.fit_lr(X_train_scaled, y_train)
lr_grid_search = mu.fit_grid_search_lr(X_train_scaled, y_train, skf)
rf_train_test = mu.fit_rf(X_train_scaled, y_train)
rf_grid_search = mu.fit_grid_search_rf(X_train_scaled, y_train, skf)
knn_train_test = mu.fit_knn(X_train_scaled, y_train)
knn_grid_search = mu.fit_grid_search_knn(X_train_scaled, y_train, skf)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 536 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:   15.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 285 out of 300 | elapsed:    1.8s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    2.0s finished


Fitting 5 folds for each of 42 candidates, totalling 210 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   28.6s
[Parallel(n_jobs=-1)]: Done 210 out of 210 | elapsed:   33.3s finished


Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   13.8s finished


In [7]:
models = {**dt_train_test, **dt_grid_serch, **lr_train_test, **lr_grid_search, **rf_train_test, **rf_grid_search,
          **knn_train_test, **knn_grid_search}
models

{'DecisionTreeClassifier_train_test': DecisionTreeClassifier(class_weight='balanced', random_state=17),
 'DecisionTreeClassifier_GridSearchCV': DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                        max_depth=3, random_state=17),
 'LogisticRegression_train_test': LogisticRegression(class_weight='balanced', random_state=17),
 'LogisticRegression_GridSearchCV': LogisticRegression(C=0.012742749857031334, class_weight='balanced',
                    random_state=17),
 'RandomForestClassifier_train_test': RandomForestClassifier(class_weight='balanced', random_state=17),
 'RandomForestClassifier_CV': RandomForestClassifier(class_weight='balanced', max_depth=10, max_features=4,
                        min_samples_leaf=9, n_estimators=55, random_state=17),
 'KNeighborsClassifier_train_test': KNeighborsClassifier(),
 'KNeighborsClassifier_CV': KNeighborsClassifier(algorithm='ball_tree', n_neighbors=8)}

In [8]:
models_performance = mu.get_models_performance(models, X_train_scaled, y_train,
                                              X_test_scaled, y_test)
models_performance

,model,train_accuracy_score,train_roc_auc_score,train_mean_cross_val_score,train_proba_roc_auc_score,test_accuracy_score,test_roc_auc_score,test_proba_roc_auc_score,test_mean_cross_val_score
0,DecisionTreeClassifier_train_test,0.971245,0.951868,0.863928,0.993191,0.881579,0.580404,0.555610,0.864159
1,DecisionTreeClassifier_GridSearchCV,0.822543,0.763598,0.798080,0.840688,0.824685,0.762208,0.834580,0.764994
2,LogisticRegression_train_test,0.750496,0.775434,0.750417,0.847352,0.743884,0.767174,0.838651,0.734425
3,LogisticRegression_GridSearchCV,0.752165,0.779501,0.759233,0.847976,0.744440,0.769690,0.840491,0.731828
4,RandomForestClassifier_train_test,0.974899,0.948438,0.902772,0.990208,0.925686,0.566639,0.778688,0.906040
5,RandomForestClassifier_CV,0.842720,0.836857,0.835809,0.917996,0.838584,0.745373,0.831397,0.863971
6,KNeighborsClassifier_train_test,0.933672,0.653395,0.913417,0.904844,0.915864,0.584575,0.716103,0.917160
7,KNeighborsClassifier_CV,0.927874,0.588404,0.920883,0.893463,0.921794,0.555673,0.763852,0.921052


In [9]:
knn_train_test_not_scaled = mu.fit_knn(X_train, y_train)
knn_grid_search_not_scaled = mu.fit_grid_search_knn(X_train, y_train, skf)
models_performance = mu.add_performance_to_df(models_performance, 'KNN_not_scaled', 
                                             list(knn_train_test_not_scaled.values())[0], X_train, y_train, X_test, y_test)
models_performance = mu.add_performance_to_df(models_performance, 'KNN_GC_not_scaled',
                                             list(knn_grid_search_not_scaled.values())[0], X_train, y_train, X_test, y_test)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   10.3s finished


In [10]:
models_performance

,model,train_accuracy_score,train_roc_auc_score,train_mean_cross_val_score,train_proba_roc_auc_score,test_accuracy_score,test_roc_auc_score,test_proba_roc_auc_score,test_mean_cross_val_score
0,DecisionTreeClassifier_train_test,0.971245,0.951868,0.863928,0.993191,0.881579,0.580404,0.555610,0.864159
1,DecisionTreeClassifier_GridSearchCV,0.822543,0.763598,0.798080,0.840688,0.824685,0.762208,0.834580,0.764994
2,LogisticRegression_train_test,0.750496,0.775434,0.750417,0.847352,0.743884,0.767174,0.838651,0.734425
3,LogisticRegression_GridSearchCV,0.752165,0.779501,0.759233,0.847976,0.744440,0.769690,0.840491,0.731828
4,RandomForestClassifier_train_test,0.974899,0.948438,0.902772,0.990208,0.925686,0.566639,0.778688,0.906040
5,RandomForestClassifier_CV,0.842720,0.836857,0.835809,0.917996,0.838584,0.745373,0.831397,0.863971
6,KNeighborsClassifier_train_test,0.933672,0.653395,0.913417,0.904844,0.915864,0.584575,0.716103,0.917160
7,KNeighborsClassifier_CV,0.927874,0.588404,0.920883,0.893463,0.921794,0.555673,0.763852,0.921052
8,KNN_not_scaled,0.934228,0.652343,0.911431,0.904526,0.914196,0.574813,0.711921,0.913637
9,KNN_GC_not_scaled,0.927556,0.580558,0.918500,0.894753,0.921053,0.545305,0.743946,0.922719


**Попробуем улучшить лучшие модели**

In [11]:
bagging_lr_gs = mu.fit_bagg_model(lr_grid_search, X_train_scaled, y_train)
bagging_dt_gs = mu.fit_bagg_model(dt_grid_serch, X_train_scaled, y_train)

In [12]:
models_performance = mu.add_performance_to_df(models_performance, 'Bagging_LR_CV', 
                                             list(bagging_lr_gs.values())[0], X_train_scaled, y_train, X_test_scaled, y_test)
models_performance = mu.add_performance_to_df(models_performance, 'Bagging_DT_CV',
                                             list(bagging_dt_gs.values())[0], X_train_scaled, y_train, X_test_scaled, y_test)

In [13]:
models_performance

,model,train_accuracy_score,train_roc_auc_score,train_mean_cross_val_score,train_proba_roc_auc_score,test_accuracy_score,test_roc_auc_score,test_proba_roc_auc_score,test_mean_cross_val_score
0,DecisionTreeClassifier_train_test,0.971245,0.951868,0.863928,0.993191,0.881579,0.580404,0.555610,0.864159
1,DecisionTreeClassifier_GridSearchCV,0.822543,0.763598,0.798080,0.840688,0.824685,0.762208,0.834580,0.764994
2,LogisticRegression_train_test,0.750496,0.775434,0.750417,0.847352,0.743884,0.767174,0.838651,0.734425
3,LogisticRegression_GridSearchCV,0.752165,0.779501,0.759233,0.847976,0.744440,0.769690,0.840491,0.731828
4,RandomForestClassifier_train_test,0.974899,0.948438,0.902772,0.990208,0.925686,0.566639,0.778688,0.906040
5,RandomForestClassifier_CV,0.842720,0.836857,0.835809,0.917996,0.838584,0.745373,0.831397,0.863971
6,KNeighborsClassifier_train_test,0.933672,0.653395,0.913417,0.904844,0.915864,0.584575,0.716103,0.917160
7,KNeighborsClassifier_CV,0.927874,0.588404,0.920883,0.893463,0.921794,0.555673,0.763852,0.921052
8,KNN_not_scaled,0.934228,0.652343,0.911431,0.904526,0.914196,0.574813,0.711921,0.913637
9,KNN_GC_not_scaled,0.927556,0.580558,0.918500,0.894753,0.921053,0.545305,0.743946,0.922719


# Тест на контрольной выборке

In [14]:
events_pred = pd.read_csv('../data/test/events_data_test.zip')
submissions_pred = pd.read_csv('../data/test/submission_data_test.zip')

In [15]:
X_pred, _ = pu.make_X_y_with_new_fetures(events_pred, submissions_pred)

In [16]:
X_pred

,days,steps_tried,correct,wrong,correct_ratio,step_correct_ratio,viewed,passed / discovered,passed / started_attempt,passed / viewed
user_id,,,,,,,,,,
12,1.0,1.0,1.0,0.0,1.000000,0.985253,14,0.953849,0.917046,0.636857
13,2.0,29.0,29.0,36.0,0.446154,0.391864,105,0.891742,0.813889,0.478873
21,1.0,30.0,24.0,103.0,0.188976,0.379718,98,0.874875,0.761897,0.465960
35,3.0,9.0,7.0,35.0,0.166667,0.528667,70,0.952976,0.914170,0.547116
45,1.0,2.0,2.0,0.0,1.000000,0.811877,10,0.962651,0.917056,0.676001
...,...,...,...,...,...,...,...,...,...,...
26743,0.0,0.0,0.0,0.0,0.000000,0.000000,1,1.000000,1.000000,0.709497
26745,0.0,0.0,0.0,0.0,0.000000,0.000000,1,1.000000,1.000000,0.709497
26768,0.0,0.0,0.0,0.0,0.000000,0.000000,1,1.000000,1.000000,0.709497


In [17]:
indexes = X_pred.index

scaler = StandardScaler()
X_pred_scaled = scaler.fit_transform(X_pred)

In [18]:
lr_gs_pred = list(lr_grid_search.values())[0].predict_proba(X_pred_scaled)[:, 1]

In [19]:
pu.write_to_csv(indexes, lr_gs_pred, '../reports/new_f_lr_gs_pred.csv')

In [20]:
from IPython import display 
display.Image(filename="../data/ROC_SCORE/new_f_LR_CV_score.png")

**Результат по метрике ROC**  
Как видим, после добавления новых признаков производительность выросла на 2 процента